In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import requests
import json
import os
from pandas.io.json import json_normalize  
import csv

In [2]:
#File Path
file_path=os.path.join("..","Final Output Data","schoolRank_data.csv")

In [3]:
#Get API key
from config import appKey

In [4]:
#URL and Parameters 
base_url="https://api.schooldigger.com/v1.2/rankings/schools/NJ?year=2019"
param="&level=Middle"
param0="&level=High"
param1="&page="
param2="&perPage=50&appID=59309557"
param3="&appKey=" + appKey

In [5]:
#School types
school_level=["Elementary","Middle","High"]

In [6]:
#Function to generate list of schools
def getSchoolList(pageCount,baseUrl):
    
    range_limit=int(pageCount)+1
    
    #Loop through all pages
    for i in range(1,range_limit):
        
        #Set page # parameter
        param1="&page="+str(i)
        
        #Build URL
        url=baseUrl+param1
    
        #Get json response
        response = requests.get(url).json()

        #Normalize json file
        df = json_normalize(response)

        #Get results from the data frame
        school_set=pd.DataFrame(df["schoolList"][0])

        # lets get address details
        school_address = json_normalize(school_set['address']) 

        #Get school address details
        try:
            street=school_address["street"]
        except:
            street=""
        try:
            city=school_address["city"]
        except:
            city=""
        try:
            zipcode=school_address["zip"]
        except:
            zipcode=""
        try:
            lat=school_address["latLong.latitude"]
        except:
            lat=""
        try:
            lng=school_address["latLong.longitude"]
        except:
            lng=""

        #Append columns in school district data set
        school_set["City"]=city
        school_set["Zip Code"]=zipcode
        school_set["Address"]=street
        school_set["Lat"]=lat
        school_set["Lng"]=lng

        #Lets work on school rank history
        school_rank=pd.DataFrame(school_set['rankHistory'][0])

        #Get school ranks - interested in 2018 and 2019 
        try:
            year2018=school_rank["year"].iloc[0]
        except:
            year2018="2018"
        try:
            year2018rank=school_rank["rank"].iloc[0]
        except:
            year2018rank=""
        try:
            year2018rankstar=school_rank["rankStars"].iloc[0]
        except:
            year2018rankstar=""
        try:
            year2018ranklevel=school_rank["rankLevel"].iloc[0]
        except:
            year2018ranklevel=""
        try:    
            year2018avgscore=school_rank["averageStandardScore"].iloc[0]
        except:
            year2018avgscore=""
        try:
            year2019=school_rank["year"].iloc[1]
        except:
            year2019="2019"
        try:
            year2019rank=school_rank["rank"].iloc[1]
        except:
            year2019rank=""
        try:    
            year2019rankstar=school_rank["rankStars"].iloc[1]
        except:
            year2019rankstar=""
        try:
            year2019ranklevel=school_rank["rankLevel"].iloc[1]
        except:
            year2019ranklevel=""
        try:
            year2019avgscore=school_rank["averageStandardScore"].iloc[1]
        except:
            year2019avgscore=""

        #Append columns in school_set
        #school_set["Year 2018"]=year2018
        school_set["2018 Rank"]=year2018rank
        school_set["2018 Rank Stars"]=year2018rankstar
        school_set["2018 Rank Level"]=year2018ranklevel
        school_set["2018 Avg Score"]=year2018avgscore
        #school_set["Year 2019"]=year2019
        school_set["2019 Rank"]=year2019rank
        school_set["2019 Rank Stars"]=year2019rankstar
        school_set["2019 Rank Level"]=year2019ranklevel
        school_set["2019 Avg Score"]=year2019avgscore

        #Get school details
        try:
            school_info=pd.DataFrame(school_set['schoolYearlyDetails'][0])

            #Add student count for school
            try:
                studentcount_2019=school_info["numberOfStudents"]
            except:
                studentcount_2019=""
        except:
            studentcount_2019=""
        finally:
            #Populate student count if available
            school_set["2019 Student Count"]=studentcount_2019

        #Populate Data 
        df_list.append(school_set)

In [7]:
#List to hold school info
df_list=[]
sURL=base_url+param2+param3

#Loop through all schools
for i in range(len(school_level)):
   
    #Get school Type
    schoolType=school_level[i]
    
    if schoolType=="Elementary":
        baseUrl=sURL
        pageCount=28

    elif schoolType=="Middle":
        baseUrl=sURL+param
        pageCount=15

    elif schoolType=="High":
        baseUrl=sURL+param0
        pageCount=9
    
    #Get school info
    getSchoolList(pageCount,baseUrl)

In [8]:
#Check number of datasets Elementtry:28,Middle School:15 and High School:9 (28+15+9=52)
len(df_list)

52

In [9]:
#Lets merge all data
school_data = pd.concat(df_list,sort=False)

In [10]:
#Check school data columns
school_data.columns

Index(['schoolid', 'schoolName', 'phone', 'url', 'urlCompare', 'address',
       'lowGrade', 'highGrade', 'schoolLevel', 'isCharterSchool',
       'isMagnetSchool', 'isVirtualSchool', 'isTitleISchool',
       'isTitleISchoolwideSchool', 'district', 'county', 'rankHistory',
       'rankMovement', 'schoolYearlyDetails', 'isPrivate', 'privateDays',
       'privateHours', 'privateHasLibrary', 'privateCoed',
       'privateOrientation', 'hasBoundary', 'City', 'Zip Code', 'Address',
       'Lat', 'Lng', '2018 Rank', '2018 Rank Stars', '2018 Rank Level',
       '2018 Avg Score', '2019 Rank', '2019 Rank Stars', '2019 Rank Level',
       '2019 Avg Score', '2019 Student Count'],
      dtype='object')

In [13]:
#Drop columns that we extracted data from
school_data_set=school_data.drop(columns=["address","county","district","hasBoundary","isCharterSchool",
                                              "isMagnetSchool","isPrivate","isTitleISchool","isTitleISchoolwideSchool",
                                              "isVirtualSchool","phone","privateCoed",
                                              "privateDays","privateHasLibrary","privateHours","privateOrientation",
                                              "rankHistory","rankMovement","schoolYearlyDetails","url","urlCompare"],axis=1)

In [14]:
#Rename columns
school_data_set = school_data_set.rename(columns ={"schoolid": "School ID","schoolName":"School Name",
                           "lowGrade":"Low Grade","highGrade":"High Grade","schoolLevel":"School Level"})

In [15]:
#Reorder column names
school_data=school_data_set[['School ID', 'School Name', 'City', 'Zip Code', 'Address', '2018 Rank', '2018 Rank Stars', 
                             '2018 Rank Level','2018 Avg Score', '2019 Rank', '2019 Rank Stars', '2019 Rank Level',
                             '2019 Avg Score','2019 Student Count', 'Low Grade', 'High Grade', 'School Level']]

In [16]:
#Check sample data
school_data.head(2)

,School ID,School Name,City,Zip Code,Address,2018 Rank,2018 Rank Stars,2018 Rank Level,2018 Avg Score,2019 Rank,2019 Rank Stars,2019 Rank Level,2019 Avg Score,2019 Student Count,Low Grade,High Grade,School Level
0,341269004924,School 28,Paterson,07522,200 Presidential Boulevard,1,5,Elementary,98.61146,1,5,Elementary,98.1334,505.0,PK,8,Elementary
1,340933000596,George Washington School,Mahwah,07430,39 Fardale Avenue,1,5,Elementary,98.61146,1,5,Elementary,98.1334,NaN,K,3,Elementary


In [17]:
#Chek if data is loaded sucessfully
school_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2512 entries, 0 to 13
Data columns (total 17 columns):
School ID             2512 non-null object
School Name           2512 non-null object
City                  2512 non-null object
Zip Code              2512 non-null object
Address               2512 non-null object
2018 Rank             2512 non-null int64
2018 Rank Stars       2512 non-null int64
2018 Rank Level       2512 non-null object
2018 Avg Score        2512 non-null float64
2019 Rank             2512 non-null object
2019 Rank Stars       2512 non-null object
2019 Rank Level       2512 non-null object
2019 Avg Score        2512 non-null object
2019 Student Count    52 non-null float64
Low Grade             2512 non-null object
High Grade            2512 non-null object
School Level          2512 non-null object
dtypes: float64(2), int64(2), object(13)
memory usage: 353.2+ KB


In [18]:
#Export data to csv
school_data.to_csv(file_path)